<a href="https://colab.research.google.com/github/imabari/covid19-data/blob/master/ibaraki/ibaraki_covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfplumber

     |████████████████████████████████| 51kB 3.0MB/s 
     |████████████████████████████████| 5.6MB 6.0MB/s 
     |████████████████████████████████| 143kB 39.2MB/s 
     |████████████████████████████████| 1.9MB 40.9MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.26-cp36-none-any.whl size=31747 sha256=5855209ef03e8052ebb3efec48f2d607d2a0b4faa516f57bbdd2712e861910a9
  Stored in directory: /root/.cache/pip/wheels/5a/4a/09/8f7b594a0d7ed1ca5fe27078a6a96e7439167a502074d87d1b
Successfully built pdfplumber


In [2]:
import datetime
import pathlib
import re
from urllib.parse import urljoin

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
import pdfplumber
import pandas as pd

In [5]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko"
}

In [6]:
def fetch_soup(url, parser="html.parser"):

    r = requests.get(url, headers=headers)
    r.raise_for_status()

    soup = BeautifulSoup(r.content, parser)

    return soup

In [7]:
def fetch_file(url, dir="."):

    p = pathlib.Path(dir, pathlib.PurePath(url).name)
    p.parent.mkdir(parents=True, exist_ok=True)
    
    r = requests.get(url)
    r.raise_for_status()

    with p.open(mode="wb") as fw:
        fw.write(r.content)
    return p

In [8]:
def pdf2df(p):

    with pdfplumber.open(p) as pdf:

        dfs = []

        for page in pdf.pages[1:]:

            table = page.extract_table()

            tmp = pd.DataFrame(table[1:], columns=table[0])

            if (tmp.columns[0] == "判明日") or (tmp.columns[1] == "判明日"):

                dfs.append(tmp)

    df = pd.concat(dfs)

    df.replace(["―", "－", ""], pd.NA, inplace=True)
    df.dropna(how="all", inplace=True)

    return df.reset_index(drop=True)

In [9]:
def str2date(s):

    df = s.str.extract("(\d{1,2})月(\d{1,2})日").rename(columns={0: "month", 1: "day"}).fillna(0).astype(int)

    df["year"] = dt_now.year

    tmp = pd.to_datetime(df, errors="coerce")

    df["year"] = df["year"].mask(tmp > dt_now, df["year"] - 1)

    return pd.to_datetime(df, errors="coerce")

In [10]:
JST = datetime.timezone(datetime.timedelta(hours=+9))
dt_now = datetime.datetime.now(JST).replace(tzinfo=None)

In [11]:
url = "https://www.pref.ibaraki.jp/1saigai/2019-ncov/hassei.html"

In [12]:
soup = fetch_soup(url)

In [13]:
pub_date = soup.select_one("div#tmp_contents > h4").get_text(strip=True).replace("発表", "")

# 茨城県

In [14]:
tag_pref = soup.find("a", class_="icon_pdf", text=re.compile("^新型コロナウイルス感染症患者の発生及び退院・退所等について"))
link_pref = urljoin(url, tag_pref.get("href"))

In [15]:
path_pref = fetch_file(link_pref)

In [16]:
df_pref = pdf2df(path_pref)

In [17]:
pub_pref = tag_pref.parent.find_previous_sibling("h4").get_text(strip=True).replace("発表", "")

In [18]:
df_pref["管轄"] = "茨城県"
df_pref["管轄ID"] = 0
df_pref["公表日"] = pub_pref

In [19]:
df_pref

,判明日,新規\n濃厚,年代,性別,職業,居住地,発症日,備考（疑われる感染経路）,管轄,管轄ID,公表日
0,2月16日,濃厚,20代,男性,会社員,取手市,2月11日,知人・家庭内,茨城県,0,2月18日
1,2月16日,濃厚,20代,男性,自営業,稲敷市,2月15日,知人・家庭内,茨城県,0,2月18日
2,2月16日,新規,30代,女性,無職,牛久市,2月16日,<NA>,茨城県,0,2月18日
3,2月16日,濃厚,20代,女性,技能実習生,河内町,2月15日,職場内,茨城県,0,2月18日
4,2月16日,濃厚,30代,女性,技能実習生,龍ヶ崎市,2月13日,職場内、知人・家庭内,茨城県,0,2月18日
5,2月16日,濃厚,20代,女性,技能実習生,龍ヶ崎市,2月13日,職場内、知人・家庭内,茨城県,0,2月18日
6,2月17日,濃厚,80代,男性,パート,美浦村,2月13日,他県、職場内,茨城県,0,2月18日
7,2月17日,濃厚,70代,男性,無職,古河市,<NA>,福祉関係,茨城県,0,2月18日
8,2月17日,濃厚,70代,女性,無職,古河市,<NA>,福祉関係,茨城県,0,2月18日
9,2月17日,濃厚,70代,女性,無職,栃木県,2月17日,医療関係,茨城県,0,2月18日


# 水戸市

In [20]:
tag_city = soup.find("a", class_="icon_pdf", text=re.compile("^【水戸市発表】新型コロナウイルス感染症患者の発生について"))
link_city = urljoin(url, tag_city.get("href"))

In [21]:
path_city = fetch_file(link_city)

In [22]:
df_city = pdf2df(path_city)

In [23]:
pub_city = tag_city.parent.find_previous_sibling("h4").get_text(strip=True).replace("発表", "")

In [24]:
df_city["管轄"] = "水戸市"
df_city["管轄ID"] = 1
df_city["公表日"] = pub_city

In [25]:
# 重複削除
df_city.drop_duplicates(inplace=True)

In [26]:
df_city

,例目,判明日,新規\n濃厚,年代,性別,職業,居住地,発症日,管轄,管轄ID,公表日
0,470例目,2月17日,濃厚,60歳代,男性,無職,水戸市,2月9日,水戸市,1,2月18日


# 結合

In [27]:
df_tmp = pd.concat([df_pref, df_city])

In [28]:
df_tmp

,判明日,新規\n濃厚,年代,性別,職業,居住地,発症日,備考（疑われる感染経路）,管轄,管轄ID,公表日,例目
0,2月16日,濃厚,20代,男性,会社員,取手市,2月11日,知人・家庭内,茨城県,0,2月18日,NaN
1,2月16日,濃厚,20代,男性,自営業,稲敷市,2月15日,知人・家庭内,茨城県,0,2月18日,NaN
2,2月16日,新規,30代,女性,無職,牛久市,2月16日,<NA>,茨城県,0,2月18日,NaN
3,2月16日,濃厚,20代,女性,技能実習生,河内町,2月15日,職場内,茨城県,0,2月18日,NaN
4,2月16日,濃厚,30代,女性,技能実習生,龍ヶ崎市,2月13日,職場内、知人・家庭内,茨城県,0,2月18日,NaN
5,2月16日,濃厚,20代,女性,技能実習生,龍ヶ崎市,2月13日,職場内、知人・家庭内,茨城県,0,2月18日,NaN
6,2月17日,濃厚,80代,男性,パート,美浦村,2月13日,他県、職場内,茨城県,0,2月18日,NaN
7,2月17日,濃厚,70代,男性,無職,古河市,<NA>,福祉関係,茨城県,0,2月18日,NaN
8,2月17日,濃厚,70代,女性,無職,古河市,<NA>,福祉関係,茨城県,0,2月18日,NaN
9,2月17日,濃厚,70代,女性,無職,栃木県,2月17日,医療関係,茨城県,0,2月18日,NaN


# 抽出

In [29]:
# 最新のみ
df = df_tmp[df_tmp["公表日"] == pub_date].copy()

# 前処理

In [30]:
# 無症状
df["状態"] = df["発症日"].where(df["発症日"] == "症状なし").replace({"症状なし": "無症状"})

In [31]:
df["年代"] = df["年代"].str.replace("歳代", "代")

In [32]:
df["性別"] = df["性別"].replace({"男子": "男性", "女子": "女性"})

In [33]:
df["職業"] = df["職業"].replace({"生徒": "学生", "小学生": "学生", "中学生": "学生", "高校生": "学生", "大学生": "学生", "非公表": ""})

In [34]:
df["患者_濃厚接触者フラグ"] = df["新規\n濃厚"].replace({"新規": 0, "濃厚": 1})

In [35]:
df["判明日"] = str2date(df["判明日"])
df["発症日"] = str2date(df["発症日"])
df["公表日"] = str2date(df["公表日"])

In [36]:
df["判明日ISO"] = df["判明日"].apply(lambda d: pd.Timestamp(d, tz=None).isoformat()).replace("NaT", "")
df["発症日ISO"] = df["発症日"].apply(lambda d: pd.Timestamp(d, tz=None).isoformat()).replace("NaT", "")
df["公表日ISO"] = df["公表日"].apply(lambda d: pd.Timestamp(d, tz=None).isoformat()).replace("NaT", "")

In [37]:
df["全国地方公共団体コード"] = "080004"
df["都道府県名"] = "茨城県"

In [38]:
# df["市区町村名"] = df["居住地"].where(df["居住地"] == "水戸市")
# df["全国地方公共団体コード"] = df["全国地方公共団体コード"].mask(df["居住地"] == "水戸市", "082015")

In [39]:
df.rename(
    columns={
        "公表日ISO": "公表_年月日",
        "発症日ISO": "発症_年月日",
        "居住地": "患者_居住地",
        "年代": "患者_年代",
        "性別": "患者_性別",
        "職業": "患者_職業",
        "状態": "患者_状態",
        "備考（疑われる感染経路）": "備考",
    },
    inplace=True,
)

In [40]:
df["備考"] = df["備考"].str.replace("、", "感染;") + "感染"

In [41]:
df1 = df.reset_index().sort_values(by=["公表日", "管轄ID", "index"]).reset_index(drop=True)

In [42]:
df1 = df1.reindex(
    [
        "全国地方公共団体コード",
        "都道府県名",
        "市区町村名",
        "公表_年月日",
        "発症_年月日",
        "患者_居住地",
        "患者_年代",
        "患者_性別",
        "患者_職業",
        "患者_状態",
        "患者_症状",
        "患者_渡航歴の有無フラグ",
        "患者_濃厚接触者フラグ",
        "検査方法",
        "備考",
    ],
    axis=1,
)

In [43]:
df1

,全国地方公共団体コード,都道府県名,市区町村名,公表_年月日,発症_年月日,患者_居住地,患者_年代,患者_性別,患者_職業,患者_状態,患者_症状,患者_渡航歴の有無フラグ,患者_濃厚接触者フラグ,検査方法,備考
0,080004,茨城県,NaN,2021-02-18T00:00:00,2021-02-11T00:00:00,取手市,20代,男性,会社員,NaN,NaN,NaN,1,NaN,知人・家庭内感染
1,080004,茨城県,NaN,2021-02-18T00:00:00,2021-02-15T00:00:00,稲敷市,20代,男性,自営業,NaN,NaN,NaN,1,NaN,知人・家庭内感染
2,080004,茨城県,NaN,2021-02-18T00:00:00,2021-02-16T00:00:00,牛久市,30代,女性,無職,NaN,NaN,NaN,0,NaN,NaN
3,080004,茨城県,NaN,2021-02-18T00:00:00,2021-02-15T00:00:00,河内町,20代,女性,技能実習生,NaN,NaN,NaN,1,NaN,職場内感染
4,080004,茨城県,NaN,2021-02-18T00:00:00,2021-02-13T00:00:00,龍ヶ崎市,30代,女性,技能実習生,NaN,NaN,NaN,1,NaN,職場内感染;知人・家庭内感染
5,080004,茨城県,NaN,2021-02-18T00:00:00,2021-02-13T00:00:00,龍ヶ崎市,20代,女性,技能実習生,NaN,NaN,NaN,1,NaN,職場内感染;知人・家庭内感染
6,080004,茨城県,NaN,2021-02-18T00:00:00,2021-02-13T00:00:00,美浦村,80代,男性,パート,NaN,NaN,NaN,1,NaN,他県感染;職場内感染
7,080004,茨城県,NaN,2021-02-18T00:00:00,,古河市,70代,男性,無職,NaN,NaN,NaN,1,NaN,福祉関係感染
8,080004,茨城県,NaN,2021-02-18T00:00:00,,古河市,70代,女性,無職,NaN,NaN,NaN,1,NaN,福祉関係感染
9,080004,茨城県,NaN,2021-02-18T00:00:00,2021-02-17T00:00:00,栃木県,70代,女性,無職,NaN,NaN,NaN,1,NaN,医療関係感染


In [44]:
df1.to_csv("080004_ibaraki_covid19_patients.csv", encoding="utf_8_sig")

In [45]:
df1.to_csv("080004_ibaraki_covid19_patients.tsv", sep="\t", encoding="utf_8_sig", index=False)

In [46]:
from google.colab import files

In [47]:
files.download("080004_ibaraki_covid19_patients.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
print(f"茨城県 {pub_pref}発表", link_pref)

茨城県 2月18日発表 https://www.pref.ibaraki.jp/1saigai/2019-ncov/documents/210218_shiryoteikyo.pdf


In [49]:
print(f"水戸市 {pub_city}発表", link_city)

水戸市 2月18日発表 https://www.pref.ibaraki.jp/1saigai/2019-ncov/documents/210218_mito.pdf
